In [ ]:
import numpy as np 
import torch 
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import math 
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import collections 


train_dataset = dsets.MNIST(root='./data',train = True,transform= transforms.ToTensor(),download=True)
show_image = train_dataset[1][0].numpy().reshape(28,28)

plt.imshow(show_image,cmap="gray")
test_dataset = dsets.MNIST(root='./data',train = False,transform= transforms.ToTensor())


batch_size  =10
n_iters = 30000
n_epoch = int(n_iters/(len(train_dataset)/batch_size))
n_epoch


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size = batch_size,shuffle =True)
isinstance(train_loader,collections.Iterable)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size =batch_size,shuffle =False)
isinstance(test_loader,collections.Iterable)

class LogisticRegresionModel(nn.Module):
  def __init__(self,input_dim,output_dim):
    super(LogisticRegresionModel,self).__init__()
    
    self.linear =nn.Linear(input_dim,output_dim).cuda()
    #self.linear = self.linear.cuda()
    
  def forward(self,x):
    out= self.linear(x)
    return out

input_dim = 28*28
output_dim =10

model = LogisticRegresionModel(input_dim,output_dim)

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

print(model.parameters())
print(len(list(model.parameters())))
print(list(model.parameters())[0].size())
print(list(model.parameters())[1].size())

iter = 0
for epoch in range(n_epoch):
  for i,(images,labels) in enumerate(train_loader):
    images = Variable(images.view(-1,28*28))
    labels = Variable(labels)

    optimizer.zero_grad()

    outputs=model(images)
    loss =criterion(outputs,labels)
    loss.backward()

    optimizer.step()

    iter+=1

    if iter %500==0:
      correct = 0
      total = 0

      for images,labels in test_loader:
        images =Variable(images.view(-1,28*28))
        outputs =model(images)

        total +=labels.size(0)

        _,predicted =torch.max(outputs.data,1)

        correct += (predicted==labels).sum()

      accuracy = 100*correct/total

      print("Iteration:{}. Loss: {}, Accuracy: {}".format(iter,loss.data,accuracy))